In [ ]:
!hf auth login --add-to-git-credential --token <hf_token>

Token is valid (permission: fineGrained).
The token `colab-hf-token` has been saved to /home/user_lerobot/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/user_lerobot/.cache/huggingface/token
Login successful.
The current active token is: `colab-hf-token`


In [ ]:
!wandb login <wandb_token>

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/user_lerobot/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!accelerate launch src/lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id={hf_user}/{dataset_name} \
  --batch_size=64\
  --steps=50000 \
  --save_freq=10000 \
  --output_dir=outputs/train/{job_name} \
  --job_name={job_name} \
  --policy.device=cuda \
  --policy.repo_id={hf_user}/{job_name} \
  --wandb.enable=true

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
config.json: 2.04kB [00:00, 52.3kB/s]
INFO 2025-08-24 06:15:21 ils/utils.py:48 Cuda backend detected, using cuda.
WARNING 2025-08-24 06:15:21 /policies.py:80 Device 'None' is not available. Switching to 'cuda'.
INFO 2025-08-24 06:15:22 ts/train.py:111 {'batch_size': 64,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                            

In [ ]:
# resume training
!accelerate launch src/lerobot/scripts/train.py \
  --steps=60000 \
  --save_freq=10000 \
  --config_path=ls <path_to_checkpoint>/pretrained_model/train_config.json \
  --wandb.enable=true \
  --resume=true

In [ ]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model